<a href="https://colab.research.google.com/github/claudiodanielpc/30DayChartChallenge/blob/main/indicadores/bienestar/indicadores_bienestar_pgd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
rm(list=ls())

# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, showtext, srvyr, haven, googledrive)

Loading required package: pacman

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘pacman’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘sysfonts’, ‘showtextdb’



showtext installed

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘minqa’, ‘numDeriv’, ‘mitools’, ‘RcppArmadillo’, ‘survey’



srvyr installed



In [2]:
#Crear directorio para guardar los archivos
dir.create("data", showWarnings = FALSE)

In [3]:
  #Conectar a google drive
drive_auth(email = "claudiodanielpc@gmail.com",cache = FALSE)

Please point your browser to the following url: 

https://accounts.google.com/o/oauth2/v2/auth?client_id=603366585132-frjlouoa3s2ono25d2l9ukvhlsrlnr7k.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&redirect_uri=https%3A%2F%2Fwww.tidyverse.org%2Fgoogle-callback%2F&response_type=code&state=5b9ae25c5030d5741b6938e209aed5e4&login_hint=claudiodanielpc%40gmail.com&access_type=offline&prompt=consent



Enter authorization code: eyJjb2RlIjoiNC8wQVNWZ2kzTHlkbWcwZHNQcjB4ZkZWc0ZIM0dGQ2pXbU41bEVWOVIybUY4VmozOW40MjI3dWRCTVg0UEJXM2hjd280Y2c2ZyIsInN0YXRlIjoiNWI5YWUyNWM1MDMwZDU3NDFiNjkzOGUyMDlhZWQ1ZTQifQ==


In [4]:
carpeta<-as_id(drive_get("https://drive.google.com/drive/folders/1TBfoik4w4DWhwFqWeZFgkZvrBdDbqzku"))


archivos<-drive_ls(carpeta)
print(archivos)

# A dribble: 3 × 3
  name             id                                drive_resource   
  <chr>            <drv_id>                          <list>           
1 pobreza_2022.dta 1T_WfYanYMOoWWbhj_CeiqxYRNrpXmDan <named list [42]>
2 pobreza_2020.dta 1TXJLNA_itWfjsGIdoj1bJQ4erNjc1wIx <named list [42]>
3 pobreza_2018.dta 1TTRfiRDGWgQwHylvduHx1IRdm5vQ3VeL <named list [42]>


In [5]:
archivos[1]

name
<chr>
pobreza_2022.dta
pobreza_2020.dta
pobreza_2018.dta


In [12]:
estimacion_pobreza <- function(file_id, file_name, variable) {
  # Descarga de datos
  local_path <- file.path("data", file_name)
  drive_download(as_id(file_id), path = local_path, overwrite = TRUE)

  # Leer datos
  datos <- read_dta(local_path)

  # Añadir año
  year <- str_extract(file_name, "\\d{4}")


  # Diseño muestral
  mydesign <- datos %>%
    as_survey_design(ids = upm,
                     strata = est_dis,
                     weights = factor)

  # Estima pobreza
  pobreza <- mydesign %>%
    #Filtrar datos para CDMX
  filter(entidad==9)%>%
    #Filtar grupo de población en pobreza
    group_by(pobreza_mmip)%>%
    summarise(
      poblacion = survey_total(vartype = "cv"),
      porcentaje = survey_prop(vartype = "cv")
    ) %>%
    mutate(porcentaje = round(porcentaje * 100, 2),
           year = as.numeric(year))%>%
  filter(pobreza_mmip == 1)


##Pobreza extrema
pobreza_extrema <- mydesign %>%
    #Filtrar datos para CDMX
  filter(entidad==9)%>%
    #Filtar grupo de población en pobreza
    group_by(pobre_ext_mmip)%>%
    summarise(
      poblacion = survey_total(vartype = "cv"),
      porcentaje = survey_prop(vartype = "cv")
    ) %>%
    mutate(porcentaje = round(porcentaje * 100, 2),
           year = as.numeric(year))%>%
  filter(pobre_ext_mmip == 1)



#Condición

if (variable=="pobreza") {
    return(pobreza)}

else if (variable=="pobreza_extrema") {

         return(pobreza_extrema)

}

else {
    stop("Variable no reconocida..")

}}

In [14]:
tabla_pobreza <- map_dfr(
  seq_len(nrow(archivos)),
  ~ estimacion_pobreza(archivos$id[.x], archivos$name[.x],variable="pobreza")
)%>%
  arrange(year)

print("Estimación de población en situación de pobreza en la CDMX")
tabla_pobreza

File downloaded:

• pobreza_2022.dta <id: 1T_WfYanYMOoWWbhj_CeiqxYRNrpXmDan>

Saved locally as:

• data/pobreza_2022.dta

File downloaded:

• pobreza_2020.dta <id: 1TXJLNA_itWfjsGIdoj1bJQ4erNjc1wIx>

Saved locally as:

• data/pobreza_2020.dta

File downloaded:

• pobreza_2018.dta <id: 1TTRfiRDGWgQwHylvduHx1IRdm5vQ3VeL>

Saved locally as:

• data/pobreza_2018.dta



[1] "Estimación de población en situación de pobreza en la CDMX"


pobreza_mmip,poblacion,poblacion_cv,porcentaje,porcentaje_cv,year
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,5957774,0.03162363,64.78,0.01996497,2018
1,5805913,0.02787769,62.83,0.01862333,2020
1,5842356,0.02846174,62.79,0.01698602,2022


In [15]:
tabla_pobreza_ext <- map_dfr(
  seq_len(nrow(archivos)),
  ~ estimacion_pobreza(archivos$id[.x], archivos$name[.x],variable="pobreza_extrema")
)%>%
  arrange(year)

print("Estimación de población en situación de pobreza extrema en la CDMX")
tabla_pobreza_ext

File downloaded:

• pobreza_2022.dta <id: 1T_WfYanYMOoWWbhj_CeiqxYRNrpXmDan>

Saved locally as:

• data/pobreza_2022.dta

File downloaded:

• pobreza_2020.dta <id: 1TXJLNA_itWfjsGIdoj1bJQ4erNjc1wIx>

Saved locally as:

• data/pobreza_2020.dta

File downloaded:

• pobreza_2018.dta <id: 1TTRfiRDGWgQwHylvduHx1IRdm5vQ3VeL>

Saved locally as:

• data/pobreza_2018.dta



[1] "Estimación de población en situación de pobreza extrema en la CDMX"


pobre_ext_mmip,poblacion,poblacion_cv,porcentaje,porcentaje_cv,year
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1719778,0.06560991,18.70,0.05824661,2018
1,1571571,0.05676477,17.01,0.05227363,2020
1,1333103,0.06789734,14.33,0.06458095,2022
